In [19]:
import pandas as pd
import sqlite3
import sys, traceback
from sklearn.impute import SimpleImputer 

In [20]:
def get_data_from_raw(db_path, table_name):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        print("successfully connected to database")
            
    except:
        print("Error in connecting database!")
        print("-"*60)
        traceback.print_exc(file=sys.stdout)
        print("-"*60)
    df = pd.read_sql('SELECT * FROM {}'.format(table_name), conn)
    conn.close()
    return df

In [21]:
df = get_data_from_raw("../database/data.db", "RAW")

successfully connected to database


In [22]:
df.head()

,unique_data,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,...,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range,RawTimeStamp
0,2549.026455,842,0,2.2,0,1,0,7,0.6,188,...,756,2549,9,7,19,0,0,1,1,2023-05-29 12:46:43
1,2631.455231,1021,1,0.5,1,0,1,53,0.7,136,...,1988,2631,17,3,7,1,1,0,2,2023-05-29 12:46:43
2,2603.736014,563,1,0.5,1,2,1,41,0.9,145,...,1716,2603,11,2,9,1,1,0,2,2023-05-29 12:46:43
3,2769.680851,615,1,2.5,0,0,0,10,0.8,131,...,1786,2769,16,8,11,1,0,0,2,2023-05-29 12:46:43
4,1411.996700,1821,1,1.2,0,13,1,44,0.6,141,...,1212,1411,8,2,15,1,1,0,1,2023-05-29 12:46:43


## Checking for missing values

In [23]:
def get_missing_values_column(df):
    cols = [x for x in df.columns]
    null_values = df.isnull().sum().to_list()

    columns_having_null_value = [cols[i] for i in range(len(cols)) if null_values[i] > 0]
    result_dict = {}
    for col in columns_having_null_value:
        result_dict[col] = df[col].dtype.name

    return result_dict

In [24]:
get_missing_values = get_missing_values_column(df)
print(get_missing_values)

{}


In [25]:
def fill_nan_value(df, fill_value, drop=False):
    cols_dtype_dict = get_missing_values_column(df)
    if drop == False:
        for cols in cols_dtype_dict.keys():
            if fill_value == "mean":
                mean = df[cols].mean()
                df[cols].fillna(mean, inplace = True)
            elif fill_value == "median":
                median = df[cols].median()
                df[cols].fillna(median, inplace = True)
            elif fill_value == "mode":
                median = df[cols].mode()[0]   
                df[cols].fillna(median, inplace = True)
    else:
        df.dropna(inplace=True)
    return df

In [26]:
df = fill_nan_value(df, fill_value="mean", drop=False)
df.isnull().sum()

unique_data      0
battery_power    0
blue             0
clock_speed      0
dual_sim         0
fc               0
four_g           0
int_memory       0
m_dep            0
mobile_wt        0
n_cores          0
pc               0
px_height        0
px_width         0
ram              0
sc_h             0
sc_w             0
talk_time        0
three_g          0
touch_screen     0
wifi             0
price_range      0
RawTimeStamp     0
dtype: int64

# Categorical Encoding

In [27]:
for i in df.columns:
    print(f"{i} -> {df[i].unique()} --> {len(df[i].unique())}")

unique_data -> [2549.02645503 2631.45523139 2603.73601399 ... 3057.53186275  869.50149254
 3919.64058355] --> 2000
battery_power -> [ 842 1021  563 ... 1139 1467  858] --> 1094
blue -> [0 1] --> 2
clock_speed -> [2.2 0.5 2.5 1.2 1.7 0.6 2.9 2.8 2.1 1.  0.9 1.1 2.6 1.4 1.6 2.7 1.3 2.3
 2.  1.8 3.  1.5 1.9 2.4 0.8 0.7] --> 26
dual_sim -> [0 1] --> 2
fc -> [ 1  0  2 13  3  4  5  7 11 12 16  6 15  8  9 10 18 17 14 19] --> 20
four_g -> [0 1] --> 2
int_memory -> [ 7 53 41 10 44 22 24  9 33 17 52 46 13 23 49 19 39 47 38  8 57 51 21  5
 60 61  6 11 50 34 20 27 42 40 64 14 63 43 16 48 12 55 36 30 45 29 58 25
  3 54 15 37 31 32  4 18  2 56 26 35 59 28 62] --> 63
m_dep -> [0.6 0.7 0.9 0.8 0.1 0.5 1.  0.3 0.4 0.2] --> 10
mobile_wt -> [188 136 145 131 141 164 139 187 174  93 182 177 159 198 185 196 121 101
  81 156 199 114 111 132 143  96 200  88 150 107 100 157 160 119  87 152
 166 110 118 162 127 109 102 104 148 180 128 134 144 168 155 165  80 138
 142  90 197 172 116  85 163 178 171 103  83 140 

In [30]:
df.max()

unique_data              3998.489196
battery_power                   1998
blue                               1
clock_speed                      3.0
dual_sim                           1
fc                                19
four_g                             1
int_memory                        64
m_dep                            1.0
mobile_wt                        200
n_cores                            8
pc                                20
px_height                       1960
px_width                        1998
ram                             3998
sc_h                              19
sc_w                              18
talk_time                         20
three_g                            1
touch_screen                       1
wifi                               1
price_range                        3
RawTimeStamp     2023-05-29 12:46:43
dtype: object

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unique_data    2000 non-null   float64
 1   battery_power  2000 non-null   int64  
 2   blue           2000 non-null   int64  
 3   clock_speed    2000 non-null   float64
 4   dual_sim       2000 non-null   int64  
 5   fc             2000 non-null   int64  
 6   four_g         2000 non-null   int64  
 7   int_memory     2000 non-null   int64  
 8   m_dep          2000 non-null   float64
 9   mobile_wt      2000 non-null   int64  
 10  n_cores        2000 non-null   int64  
 11  pc             2000 non-null   int64  
 12  px_height      2000 non-null   int64  
 13  px_width       2000 non-null   int64  
 14  ram            2000 non-null   int64  
 15  sc_h           2000 non-null   int64  
 16  sc_w           2000 non-null   int64  
 17  talk_time      2000 non-null   int64  
 18  three_g 